# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import os
import glob
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# Check current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

c:\Users\jacob\Documents\data_engineering_nanodegree\sparkify-nosql


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# Initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# For every filepath in the file path list 
for f in file_path_list:
    # Read csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # Create a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader) # Skip header row
        # Extract each data row append it to full_data_rows_list        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
print(len(full_data_rows_list)) # Print # of rows

# Create a smaller event data csv file called event_datafile_new.csv that will be used to insert data into the
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)
with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

8056


In [4]:
# Check the number of rows in the new csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkifynosql 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkifynosql')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Query 1

**Query**: Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4.

**Table design description**: Based on this query description, the end user needs to be able to filter on sessionId and itemInSession, and retrieve a few other columns. As a result, pairing the two (i.e., sessionId and itemInSession) as the primary key makes sense as it creates a unique identifier and allows the end user to use the two values in the WHERE clause.

In [8]:
## Query 1: Give me the artist, song title and song's length in the music app history that was heard during
## sessionId = 338, and itemInSession = 4
query = "CREATE TABLE IF NOT EXISTS sparkify_session_item_search "
query = query + "(artist text, song text, length decimal, sessionId int, itemInSession int, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [10]:
## Loop through the data file and insert the rows to the table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO sparkify_session_item_search (artist, song, length, sessionId, itemInSession) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))

In [11]:
## Verify the data was entered into the table with a SELECT statement
query = "SELECT artist, song, length FROM sparkify_session_item_search WHERE sessionId=338 AND itemInSession=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.3073


## Query 2

**Query**: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182.

**Table design description**: Based on this query description, the end user needs to be able to filter on sessionId and userId, retrieve a few other columns, and order the results in a particular manner. To be able to filter on userId and sessionId, those were the columns I decided to start with to use as the primary key. When glancing at the data, though, it's easy to see that these two columns alone don't provide a unqique identifier. On top of that, we need to make sure to sort by itemInSession. Because clustering columns sort in the order they are added to the primary key, adding itemInSession to the end of the primary key makes the primary key both unique and sort in the correct order.

In [12]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)
## for userid = 10, sessionid = 182
query = "CREATE TABLE IF NOT EXISTS sparkify_user_session_search "
query = query + "(artist text, song text, sessionId int, itemInSession int, userId int, firstName text, lastName text, PRIMARY KEY (sessionId, userId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [13]:
## Loop through the data file and insert the rows to the table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO sparkify_user_session_search (artist, song, sessionId, itemInSession, userId, firstName, lastName) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9], int(line[8]), int(line[3]), int(line[10]), line[1], line[4]))

In [14]:
## Verify the data was entered into the table with a SELECT statement
query = "SELECT artist, song, firstName, lastName FROM sparkify_user_session_search WHERE sessionId=182 AND userId=10"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


## Query 3

**Query**: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'.

**Table design description**: Based on this query description, the end user needs to be able to filter on the title of the song, and retrieve the user information. The song title alone, however, is not unique and shouldn't be used in isolation as a primary key. To make it unique, then, adding the userId as a clustering column makes the primary key unique and allows the end user to search for specific songs.

*As a side note, I thought about adding in a different or additional clustering column to make the primary key unique because I could see a world where a userId and song title were not unique on their own. After all, I listen to the same song more than once in different sessions. However, in this dataset, the two made a unique pair, so I decided to leave it for simplicity.*

In [15]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = "CREATE TABLE IF NOT EXISTS sparkify_song_search "
query = query + "(firstName text, lastName text, song text, userId int, PRIMARY KEY (song, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [16]:
## Loop through the data file and insert the rows to the table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO sparkify_song_search (firstName, lastName, song, userId) "
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[1], line[4], line[9], int(line[10])))

In [17]:
## Verify the data was entered into the table with a SELECT statement
query = "SELECT firstName, lastName FROM sparkify_song_search WHERE song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [18]:
## Drop the tables before closing out the sessions
query = "DROP TABLE IF EXISTS sparkify_session_item_search"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE IF EXISTS sparkify_user_session_search"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE IF EXISTS sparkify_song_search"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()